In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
from symsplash.collapse import NFWHalo, symmetron, SSHalo
import matplotlib.ticker as mticker
from scipy.interpolate import interp1d

#plt.rc('font', size=18)  
plt.style.use('fast')

DENSITY PROFILE
========

In [ ]:
plt.figure(figsize=(3.5, 4))

SS = SSHalo(z_ssb=1., beta=1.)
SSs = SSHalo(z_ssb=1., beta=1., load_dir='cache_smooth/')

y = np.geomspace(5e-2, 1, 1000)

fac = 3*np.pi*np.pi/4.

line = plt.loglog(y, fac*SS.P(y), c='#d62728')
plt.loglog(y, fac*SSs.P(y), ls=':', c=line[0].get_color(), label='Smoothed')


ax = plt.gca()
ax.set_xlabel('Radial coordinate \n $r/R(t)$')
ax.set_ylabel('Matter density \n $\\rho(r, t) / \\rho_m(t)$')


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
ax.xaxis.set_major_formatter(mticker.ScalarFormatter())

plt.legend(loc='best', frameon=False)

plt.tight_layout()
plt.grid('on', which='both', alpha=0.1)
plt.savefig("output/figures/1.pdf")


PS DISTRIBUTION AND FORCE
================

In [ ]:
taus = np.linspace(0.1, 1, 3000)
taus_mark = np.geomspace(0.1, 1, 8)
taus_int = np.linspace(0.4, 1, 200)
r_R = np.linspace(0.05, 1, 100)
zs = taus**(-2./3.)-1.


SSs = SSHalo(z_ssb=2., beta=3., load_dir='cache/', size=20)
positions = SSs.pos(taus)
sym = symmetron(SSs, ts=taus_mark)
positions_changed = SSs.pos(taus_int, symmetron=sym)

x_x = positions_changed[:, 0]
x_p = positions_changed[:, 1]
r_sp_p = x_x[x_p>0].max()

x_x = positions[:, 0]
x_p = positions[:, 1]
r_sp =  x_x[x_p>0].max()

In [ ]:
fig, axes = plt.subplots(ncols=2, sharex=True, figsize=(8,3.5))
fig.subplots_adjust(hspace=0)
ax1, ax2 = axes


norm = mpl.colors.Normalize(vmin=zs.min(), vmax=zs.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.inferno)




sp = ax1.scatter(positions[:, 0], positions[:, 1], c=zs, cmap="inferno", s=10.)
ax1.plot(positions_changed[:, 0], positions_changed[:, 1], c='w', lw=2)
ax1.plot(positions_changed[:, 0], positions_changed[:, 1], ls=':', c='k', lw=2)


ax1.arrow(r_sp, -3.5, 0, -1., color='k', lw=1, head_length=0.5, head_width=0.02)
ax1.arrow(r_sp_p, -3.5, 0, -1., color='k', lw=1, hatch='-', head_length=0.5, head_width=0.02)
#ax1.plot([r_sp, r_sp], [-6, -3], c='k', lw=1)
ax1.plot([r_sp_p, r_sp_p], [-3.5, -4.25], c='w', ls=':', lw=2)


for tau in taus_mark:
    z = tau**(-2./3.)-1.
    print z
    lambdas = r_R * SSs.Lambda(tau)
    F_N = SSs.f0 * SSs.M(r_R) * tau**(2./3.*SSs.s)/lambdas**2.
    F_S = SSs.f1 * sym.force(r_R,tau)/SSs.Lambda(tau)
    ax2.plot(r_R, F_S/F_N, c=cmap.to_rgba(z) )



ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
#ax2.xaxis.tick_top()
ax2.set_xticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax2.set_xlim([0.05, 1])
ax1.set_ylim([-5, 5])
ax1.grid('on', which='both', alpha=0.1)
ax2.grid('on', which='both', alpha=0.1)


ax1.set_xlabel('Present-day position\n $r/R(t_0)$')
ax1.set_ylabel('Present-day velocity\n $ \\dot{r}$ $t_0/R(t_0) $')

ax2.set_xlabel('Position\n $r/R(t)$')
ax2.set_ylabel('Force ratio\n $F_\\varphi/F_N$')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.89, 0.1, 0.03, 0.8])
colorbar = fig.colorbar(sp, cax=cbar_ax)
colorbar.set_label("Redshift")


plt.tight_layout(rect=[0, 0, 0.89, 1])
plt.savefig('output/figures/2.pdf')

MONEY PLOT f (z color)
=======

In [ ]:
f_u = np.linspace(1e-2, 1, 10)
z_ssb_u = np.linspace(1, 3, 5)
size_u = np.linspace(10, 50, 5)

rspSS = np.load("output/rspSS.npy")
rspSSss = np.load("output/SS_smooth/rsp.npy")
resSSs = (rspSSss - rspSS)/rspSS


f, z_ssb, size = np.meshgrid(f_u, z_ssb_u, size_u)
f, z_ssb, size = f.flatten(), z_ssb.flatten(), size.flatten()
beta = np.sqrt(size**2.*f/(1+z_ssb)**3.)
colors = ['k', 'r', 'b', 'y', 'g']
markers = ['x', 'o', '*', '.', 'x']
beta_u = np.unique(beta)



norm = mpl.colors.Normalize(vmin=z_ssb_u.min(), vmax=z_ssb_u.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.inferno)
cmap.set_array([])

plt.figure(figsize=(8, 3.5))


for i in xrange(len(z_ssb_u)):
    y_max = np.zeros(len(f_u))
    y_min = 100*np.ones(len(f_u))
    
    for j in xrange(len(f_u)):
        for k in xrange(len(size_u)):
            idx_temp = (z_ssb == z_ssb_u[i]) & (f == f_u[j]) & (size == size_u[k])
            if(resSSs[idx_temp]>y_max[j]):
                    y_max[j] = resSSs[idx_temp]
            if(resSSs[idx_temp]<y_min[j]):
                    y_min[j] = resSSs[idx_temp]

    x = f_u
    plt.fill_between(x, y_max, y_min, color=cmap.to_rgba(z_ssb_u[i]), alpha=1.)

#plt.gca().set_xlim([0.01, 100])
plt.gca().set_ylim([0.01, 0.15])
plt.gca().set_xlabel('Dimensionless force parameter \n $f$')
plt.gca().set_ylabel('Fractional change \n $\\Delta r_\\mathrm{sp} / r_\\mathrm{sp}$')
colormap = plt.colorbar(cmap, aspect=7)
colormap.set_label('SSB redshift\n $z_\mathrm{ssb}$')
plt.grid('on', which='both', alpha=0.1)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

vals = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in vals])
plt.tight_layout()
plt.savefig("output/figures/3.pdf")

MONEY PLOT z (f labels)
=================

In [ ]:
f_u = np.linspace(1e-1, 1, 4)
z_ssb_u = np.linspace(1, 3, 20)
size_u = np.linspace(10, 50, 5)

rspSS = np.load("output/rspSS.npy")
rspSSss = np.load("output/SS_smooth_z/rsp.npy")
resSSs = (rspSSss - rspSS)/rspSS


f, z_ssb, size = np.meshgrid(f_u, z_ssb_u, size_u)
f, z_ssb, size = f.flatten(), z_ssb.flatten(), size.flatten()
beta = np.sqrt(size**2.*f/(1+z_ssb)**3.)
colors = ['k', 'r', 'b', 'y', 'g']
markers = ['x', 'o', '*', '.', 'x']
beta_u = np.unique(beta)
x = z_ssb_u
x_i = np.linspace(z_ssb_u.min(), z_ssb_u.max(), 100)

norm = mpl.colors.Normalize(vmin=f_u.min(), vmax=f_u.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.viridis)
cmap.set_array([])

plt.figure(figsize=(8, 3.5))

ls=['--', '-']


for i in xrange(len(f_u)):
    y_max = np.zeros(len(z_ssb_u))
    y_min = 100*np.ones(len(z_ssb_u))
    
    for j in xrange(len(z_ssb_u)):
        for k in xrange(len(size_u)):
            idx_temp = (z_ssb == z_ssb_u[j]) & (f == f_u[i]) & (size == size_u[k])
            if(resSSs[idx_temp]>y_max[j]):
                    y_max[j] = resSSs[idx_temp]
            if(resSSs[idx_temp]<y_min[j]):
                    y_min[j] = resSSs[idx_temp]
    

    for k in [0, -1]:
        plt.plot(x_i,interp1d(x, resSSs[ (f==f_u[i]) & (size==size_u[k])], kind='cubic')(x_i), ls=ls[k], c='k', alpha=0.3)
        #plt.text(pos_text[k][0], pos_text[k][1], text[k], rotation=rotation[k])
        if(i==0):
            plt.plot(x_i,interp1d(x, resSSs[ (f==f_u[i]) & (size==size_u[k])], kind='cubic')(x_i), ls=ls[k], c='k', alpha=0.3, 
                     label="${:.2}$".format(1./size_u[k]))
    y_min_i = interp1d(x, y_min, kind='cubic')(x_i)
    y_max_i = interp1d(x, y_max, kind='cubic')(x_i)
    
    plt.fill_between(x_i, y_max_i, y_min_i, color=cmap.to_rgba(f_u[i]), alpha=0.7, label="${:.2}$".format(f_u[i])) #label="$f ={:.2}$".format(f_u[i]))

plt.gca().set_xlim([1, 3])
plt.gca().set_ylim([0.01, 0.19])
plt.gca().set_xlabel('SSB redshift \n $z_{ssb}$')
plt.gca().set_ylabel('Fractional change \n $\\Delta r_\\mathrm{sp} / r_\\mathrm{sp}$')

plt.plot([1, 1], [0.02, 0.02], label=" ", alpha=0)
plt.plot([1, 1], [0.02, 0.02], label=" ", alpha=0)

handles, labels = plt.gca().get_legend_handles_labels()
order = [4, 5, 2, 0, 6, 7, 3, 1]
legend1 = plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], frameon=False, fontsize=12, ncol=2, title="$f$")

#order = [0, 1]
#plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], frameon=False, fontsize=12, ncol=2, bbox_to_anchor=(0.95, 0.2))
#plt.gca().add_artist(legend1)

#plt.text(2.5, 0.16, "$f$", size=12)
plt.text(2.61, 0.1215, "$\lambda_0/R(t_0)$", size=10)


plt.grid('on', which='both', alpha=0.1)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

vals = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in vals])
plt.tight_layout()
plt.savefig("output/figures/3.pdf")